In [ ]:
# -*- coding: utf-8 -*-
"""
경량형 온디바이스 한국어 TTS 모델 (Tacotron2 기반)
JSON 파일의 TransLabelText를 활용한 텍스트 처리 및 학습
"""

import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import librosa
import numpy as np
from typing import List, Dict, Tuple, Optional
import re
from jamo import h2j, j2hcj
import pickle

# 한국어 텍스트 전처리 클래스
class KoreanTextProcessor:
    def __init__(self):
        # 한국어 자모 분리를 위한 매핑
        self.char_to_id = {}
        self.id_to_char = {}
        self._build_vocab()
    
    # KoreanTextProcessor.__init__ 수정
    def _build_vocab(self):
        chars = ['<PAD>', '<START>', '<END>', ' ', '!', '?', '.', ',', ';', ':', '-', '(', ')']
        cho = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 
            'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
        jung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ',
                'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
        jong = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ',
                'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ',
                'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
        
        all_chars = chars + cho + jung + jong
        
        for i, char in enumerate(all_chars):
            self.char_to_id[char] = i
            self.id_to_char[i] = char
        
        print(f"어휘 크기: {len(all_chars)}")
    
    def korean_to_jamo(self, text: str) -> str:
        """한글을 자모로 분리"""
        result = []
        for char in text:
            if '가' <= char <= '힣':  # 한글인 경우
                decomposed = j2hcj(h2j(char))
                result.append(decomposed)
            else:
                result.append(char)
        return ''.join(result)
    
    def normalize_text(self, text: str) -> str:
        """텍스트 정규화"""
        # 특수문자 정리
        text = re.sub(r'[^\w\s가-힣!?.,;:]', '', text)
        # 공백 정리
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def text_to_sequence(self, text: str) -> List[int]:
        text = self.normalize_text(text)
        text = self.korean_to_jamo(text)
        
        sequence = [1]  # START 토큰 (<START> = 1)
        max_vocab_id = len(self.char_to_id) - 1
        
        for char in text:
            char_id = self.char_to_id.get(char, 0)  # PAD = 0
            char_id = min(char_id, max_vocab_id)
            sequence.append(char_id)
        
        sequence.append(2)  # END 토큰 (<END> = 2)
        return sequence
    
    def sequence_to_text(self, sequence: List[int]) -> str:
        """시퀀스를 텍스트로 변환"""
        text = []
        for id in sequence:
            if id in self.id_to_char:
                char = self.id_to_char[id]
                if char not in ['<PAD>', '<START>', '<END>']:
                    text.append(char)
        return ''.join(text)

# JSON 데이터셋 클래스
class KoreanTTSDataset(Dataset):
    def __init__(self, json_dir: str, audio_dir: str, text_processor: KoreanTextProcessor):
        self.text_processor = text_processor
        self.audio_dir = audio_dir
        self.data = []
        
        # 디렉토리에서 모든 JSON 파일 찾기
        json_files = self._find_json_files(json_dir)
        print(f"찾은 JSON 파일 수: {len(json_files):,}")
        
        # JSON 파일들에서 데이터 로드 (진행률 표시)
        print("🔄 JSON 파일 처리 중...")
        total_files = len(json_files)
        
        for i, json_file in enumerate(json_files):
            self._load_json_data(json_file)
            
           # 진행률 표시 (500개마다 또는 5%마다) - 원래 1000개에서 500개로 변경
            if (i + 1) % 500 == 0 or (i + 1) % max(1, total_files // 20) == 0:
                progress = (i + 1) / total_files * 100
                print(f"진행률: {i+1:,}/{total_files:,} ({progress:.1f}%) - 유효 데이터: {len(self.data):,}개")
        
        print(f"✅ JSON 파일 처리 완료: 총 {len(self.data):,}개 데이터 로드")
    
    def _find_json_files(self, json_dir: str) -> List[str]:
        """디렉토리에서 모든 JSON 파일을 재귀적으로 찾기"""
        json_files = []
        
        if not os.path.exists(json_dir):
            print(f"경고: 디렉토리가 존재하지 않습니다: {json_dir}")
            return json_files
        
        print("📂 JSON 파일 스캔 중...")
        
        # os.walk()를 사용하여 하위 디렉토리까지 모든 JSON 파일 찾기
        for root, dirs, files in os.walk(json_dir):
            for file in files:
                if file.lower().endswith('.json'):
                    full_path = os.path.join(root, file)
                    json_files.append(full_path)
                    if len(json_files) % 1000 == 0:  # 1000개마다 진행상황 표시
                        print(f"📄 JSON 파일 스캔 중... {len(json_files):,}개 발견")
        
        print(f"✅ JSON 파일 스캔 완료: {len(json_files):,}개")
        return sorted(json_files)
    
    def _load_json_data(self, json_file: str):
        """JSON 파일에서 TransLabelText 데이터 로드 (다양한 구조 지원)"""
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            trans_label_text = None
            
            # "전사정보" 키에서 TransLabelText 찾기 (우선순위 1)
            if isinstance(data, dict) and '전사정보' in data:
                if isinstance(data['전사정보'], dict) and 'TransLabelText' in data['전사정보']:
                    trans_label_text = data['전사정보']['TransLabelText']
                    
            # 만약 위에서 못 찾았다면 다른 구조도 확인
            if not trans_label_text:
                if isinstance(data, dict):
                    # 직접 TransLabelText가 있는 경우
                    if 'TransLabelText' in data:
                        trans_label_text = data['TransLabelText']
                    else:
                        # 재귀적으로 찾기
                        trans_label_text = self._find_trans_label_text_recursive(data)
                elif isinstance(data, list):
                    # 리스트 형태인 경우
                    for item in data:
                        if isinstance(item, dict) and '전사정보' in item:
                            if isinstance(item['전사정보'], dict) and 'TransLabelText' in item['전사정보']:
                                trans_label_text = item['전사정보']['TransLabelText']
                                break
            
            # 오디오 파일 경로 찾기 (JSON 파일명 기반)
            if trans_label_text and isinstance(trans_label_text, str) and trans_label_text.strip():
                audio_path = self._find_audio_path_improved(json_file)
                
                self.data.append({
                    'text': trans_label_text.strip(),
                    'audio': audio_path,
                    'json_file': json_file
                })
                
                # 처음 5개 파일만 상태 출력
                if len(self.data) <= 5:
                    if audio_path and os.path.exists(audio_path):
                        print(f"✅ 매칭 성공: {os.path.basename(json_file)} -> {os.path.basename(audio_path)}")
                    else:
                        print(f"❌ 오디오 없음: {os.path.basename(json_file)} -> {audio_path}")
                        
        except Exception as e:
            if len(self.data) < 3:  # 처음 3개만 오류 출력
                print(f"JSON 파일 로딩 오류 {os.path.basename(json_file)}: {e}")
    
    def _find_trans_label_text_recursive(self, data, max_depth=2, current_depth=0):
        """재귀적으로 TransLabelText 찾기 (깊이 제한)"""
        if current_depth >= max_depth:
            return None
            
        if isinstance(data, dict):
            # 우선 "전사정보" 키 확인
            if '전사정보' in data and isinstance(data['전사정보'], dict):
                if 'TransLabelText' in data['전사정보']:
                    return data['전사정보']['TransLabelText']
            
            # 직접 TransLabelText 키가 있는지 확인
            if 'TransLabelText' in data:
                return data['TransLabelText']
            
            # 다른 키들을 재귀적으로 탐색 (깊이 제한)
            for key, value in data.items():
                if key != '기본정보' and isinstance(value, (dict, list)):  # 기본정보는 제외
                    result = self._find_trans_label_text_recursive(value, max_depth, current_depth + 1)
                    if result:
                        return result
        
        elif isinstance(data, list):
            for item in data[:3]:  # 리스트의 처음 3개만 확인
                if isinstance(item, (dict, list)):
                    result = self._find_trans_label_text_recursive(item, max_depth, current_depth + 1)
                    if result:
                        return result
        
        return None
    
    def _find_audio_path_improved(self, json_file: str):
        """개선된 오디오 파일 경로 찾기"""
        # JSON 파일명에서 확장자 제거
        base_name = os.path.splitext(os.path.basename(json_file))[0]
        json_dir = os.path.dirname(json_file)
        
        # 1. JSON 파일과 같은 디렉토리에서 .wav 파일 찾기
        wav_path = os.path.join(json_dir, base_name + '.wav')
        if os.path.exists(wav_path):
            return wav_path
        
        # 2. 라벨링데이터 -> 원천데이터로 경로 변경
        if '라벨링데이터' in json_dir:
            audio_dir = json_dir.replace('라벨링데이터', '원천데이터')
            # TL22 -> TS22 변경
            if 'TL22' in audio_dir:
                audio_dir = audio_dir.replace('TL22', 'TS22')
            
            wav_path = os.path.join(audio_dir, base_name + '.wav')
            if os.path.exists(wav_path):
                return wav_path
        
        # 3. 직접 지정된 audio_dir 사용 (만약 설정되어 있다면)
        if hasattr(self, 'audio_dir') and self.audio_dir:
            # JSON의 상대 경로 구조를 audio_dir에 적용
            json_relative_parts = json_dir.split(os.sep)
            # 0001_G2A2E7_KMJ 같은 폴더명 찾기
            speaker_folder = None
            for part in json_relative_parts:
                if '_G2A2E7_' in part:  # 화자 폴더 패턴
                    speaker_folder = part
                    break
            
            if speaker_folder:
                wav_path = os.path.join(self.audio_dir, speaker_folder, base_name + '.wav')
                if os.path.exists(wav_path):
                    return wav_path
        
        return None
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        text = item['text']
        text_sequence = self.text_processor.text_to_sequence(text)
        
        # 오디오 경로가 이미 전체 경로임
        mel_spectrogram = self._process_audio(item['audio'])  # self.audio_dir 제거
        
        return {
            'text': torch.LongTensor(text_sequence),
            'mel': torch.FloatTensor(mel_spectrogram),
            'text_length': len(text_sequence),
            'mel_length': mel_spectrogram.shape[1]
        }
    
    def _process_audio(self, audio_path: str) -> np.ndarray:
        """오디오 파일을 멜 스펙트로그램으로 변환"""
        try:
            # 오디오 로드
            audio, sr = librosa.load(audio_path, sr=22050)
            
            # 멜 스펙트로그램 생성
            mel = librosa.feature.melspectrogram(
                y=audio,
                sr=sr,
                n_fft=1024,
                hop_length=256,
                win_length=1024,
                n_mels=80,
                fmin=0,
                fmax=8000
            )
            
            # 로그 스케일 변환
            mel = np.log(mel + 1e-9)
            
            return mel
            
        except Exception as e:
            print(f"오디오 처리 오류 {audio_path}: {e}")
            # 더미 데이터 반환
            return np.zeros((80, 100))

# Tacotron2 모델 구성요소들
class LocationLayer(nn.Module):
    def __init__(self, attention_n_filters, attention_kernel_size, attention_dim):
        super(LocationLayer, self).__init__()
        padding = int((attention_kernel_size - 1) / 2)
        self.location_conv = nn.Conv1d(2, attention_n_filters, kernel_size=attention_kernel_size, padding=padding, bias=False)
        self.location_dense = nn.Linear(attention_n_filters, attention_dim, bias=False)
    
    def forward(self, attention_weights_cat):
        processed_attention = self.location_conv(attention_weights_cat)
        processed_attention = processed_attention.transpose(1, 2)
        processed_attention = self.location_dense(processed_attention)
        return processed_attention

class Attention(nn.Module):
    def __init__(self, attention_rnn_dim, embedding_dim, attention_dim, attention_location_n_filters, attention_location_kernel_size):
        super(Attention, self).__init__()
        self.query_layer = nn.Linear(attention_rnn_dim, attention_dim, bias=False)
        self.memory_layer = nn.Linear(embedding_dim, attention_dim, bias=False)
        self.v = nn.Linear(attention_dim, 1, bias=False)
        self.location_layer = LocationLayer(attention_location_n_filters, attention_location_kernel_size, attention_dim)
        self.score_mask_value = -float("inf")
    
    def get_alignment_energies(self, query, processed_memory, attention_weights_cat):
        processed_query = self.query_layer(query.unsqueeze(1))
        processed_attention_weights = self.location_layer(attention_weights_cat)
        energies = self.v(torch.tanh(processed_query + processed_attention_weights + processed_memory))
        energies = energies.squeeze(-1)
        return energies
    
    def forward(self, attention_hidden_state, memory, processed_memory, attention_weights_cat, mask):
        alignment = self.get_alignment_energies(attention_hidden_state, processed_memory, attention_weights_cat)
        
        if mask is not None:
            alignment.data.masked_fill_(mask, self.score_mask_value)
        
        attention_weights = F.softmax(alignment, dim=1)
        attention_context = torch.bmm(attention_weights.unsqueeze(1), memory)
        attention_context = attention_context.squeeze(1)
        
        return attention_context, attention_weights

class Prenet(nn.Module):
    def __init__(self, in_dim, sizes):
        super(Prenet, self).__init__()
        in_sizes = [in_dim] + sizes[:-1]
        self.layers = nn.ModuleList([nn.Linear(in_size, out_size) for (in_size, out_size) in zip(in_sizes, sizes)])
    
    def forward(self, x):
        for linear in self.layers:
            x = F.dropout(F.relu(linear(x)), p=0.5, training=True)
        return x

class Postnet(nn.Module):
    def __init__(self, mel_dim, postnet_embedding_dim, postnet_kernel_size, postnet_n_convolutions):
        super(Postnet, self).__init__()
        self.convolutions = nn.ModuleList()
        
        self.convolutions.append(
            nn.Sequential(
                nn.Conv1d(mel_dim, postnet_embedding_dim, kernel_size=postnet_kernel_size, stride=1, padding=int((postnet_kernel_size - 1) / 2), dilation=1, bias=False),
                nn.BatchNorm1d(postnet_embedding_dim)
            )
        )
        
        for i in range(1, postnet_n_convolutions - 1):
            self.convolutions.append(
                nn.Sequential(
                    nn.Conv1d(postnet_embedding_dim, postnet_embedding_dim, kernel_size=postnet_kernel_size, stride=1, padding=int((postnet_kernel_size - 1) / 2), dilation=1, bias=False),
                    nn.BatchNorm1d(postnet_embedding_dim)
                )
            )
        
        self.convolutions.append(
            nn.Sequential(
                nn.Conv1d(postnet_embedding_dim, mel_dim, kernel_size=postnet_kernel_size, stride=1, padding=int((postnet_kernel_size - 1) / 2), dilation=1, bias=False),
                nn.BatchNorm1d(mel_dim)
            )
        )
    
    def forward(self, x):
        for i in range(len(self.convolutions) - 1):
            x = F.dropout(torch.tanh(self.convolutions[i](x)), 0.5, self.training)
        x = F.dropout(self.convolutions[-1](x), 0.5, self.training)
        return x

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, encoder_n_convolutions, encoder_embedding_dim, encoder_kernel_size):
        super(Encoder, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        std = np.sqrt(2.0 / (vocab_size + embedding_dim))
        val = np.sqrt(3.0) * std
        self.embedding.weight.data.uniform_(-val, val)
        
        convolutions = []
        for _ in range(encoder_n_convolutions):
            conv_layer = nn.Sequential(
                nn.Conv1d(embedding_dim, encoder_embedding_dim, kernel_size=encoder_kernel_size, stride=1, padding=int((encoder_kernel_size - 1) / 2), dilation=1, bias=False),
                nn.BatchNorm1d(encoder_embedding_dim),
                nn.ReLU(),
                nn.Dropout(0.5)
            )
            convolutions.append(conv_layer)
            embedding_dim = encoder_embedding_dim
        
        self.convolutions = nn.ModuleList(convolutions)
        self.lstm = nn.LSTM(encoder_embedding_dim, int(encoder_embedding_dim // 2), 1, batch_first=True, bidirectional=True)
    
    def forward(self, x, input_lengths):
        x = self.embedding(x).transpose(1, 2)
        
        for conv in self.convolutions:
            x = conv(x)
        
        x = x.transpose(1, 2)
        
        input_lengths = input_lengths.cpu().numpy()
        x = nn.utils.rnn.pack_padded_sequence(x, input_lengths, batch_first=True, enforce_sorted=False)
        
        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(x)
        
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        
        return outputs
    
    def inference(self, x):
        x = self.embedding(x).transpose(1, 2)
        
        for conv in self.convolutions:
            x = conv(x)
        
        x = x.transpose(1, 2)
        
        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(x)
        
        return outputs

class Decoder(nn.Module):
    def __init__(self, mel_dim, encoder_embedding_dim, attention_rnn_dim, decoder_rnn_dim, attention_dim, attention_location_n_filters, attention_location_kernel_size, prenet_dim, max_decoder_steps, gate_threshold, p_attention_dropout, p_decoder_dropout):
        super(Decoder, self).__init__()
        self.mel_dim = mel_dim
        self.encoder_embedding_dim = encoder_embedding_dim
        self.attention_rnn_dim = attention_rnn_dim
        self.decoder_rnn_dim = decoder_rnn_dim
        self.attention_dim = attention_dim
        self.attention_location_n_filters = attention_location_n_filters
        self.attention_location_kernel_size = attention_location_kernel_size
        self.prenet_dim = prenet_dim
        self.max_decoder_steps = max_decoder_steps
        self.gate_threshold = gate_threshold
        self.p_attention_dropout = p_attention_dropout
        self.p_decoder_dropout = p_decoder_dropout
        
        self.prenet = Prenet(mel_dim, [prenet_dim, prenet_dim])
        
        self.attention_rnn = nn.LSTMCell(prenet_dim + encoder_embedding_dim, attention_rnn_dim)
        
        self.attention_layer = Attention(attention_rnn_dim, encoder_embedding_dim, attention_dim, attention_location_n_filters, attention_location_kernel_size)
        
        self.decoder_rnn = nn.LSTMCell(attention_rnn_dim + encoder_embedding_dim, decoder_rnn_dim, 1)
        
        self.linear_projection = nn.Linear(decoder_rnn_dim + encoder_embedding_dim, mel_dim)
        
        self.gate_layer = nn.Linear(decoder_rnn_dim + encoder_embedding_dim, 1, bias=True)
    
    def get_go_frame(self, memory):
        B = memory.size(0)
        go_frame = torch.zeros(B, self.mel_dim, device=memory.device, dtype=memory.dtype)
        return go_frame
    
    # def initialize_decoder_states(self, memory, mask):
    #     B = memory.size(0)
    #     MAX_TIME = memory.size(1)
        
    #     attention_hidden = torch.zeros(B, self.attention_rnn_dim, device=memory.device, dtype=memory.dtype)
    #     attention_cell = torch.zeros(B, self.attention_rnn_dim, device=memory.device, dtype=memory.dtype)
        
    #     decoder_hidden = torch.zeros(B, self.decoder_rnn_dim, device=memory.device, dtype=memory.dtype)
    #     decoder_cell = torch.zeros(B, self.decoder_rnn_dim, device=memory.device, dtype=memory.dtype)
        
    #     attention_weights = torch.zeros(B, MAX_TIME, device=memory.device, dtype=memory.dtype)
    #     attention_weights_cum = torch.zeros(B, MAX_TIME, device=memory.device, dtype=memory.dtype)
    #     attention_context = torch.zeros(B, self.encoder_embedding_dim, device=memory.device, dtype=memory.dtype)
        
    #     return (attention_hidden, attention_cell, decoder_hidden, decoder_cell, attention_weights, attention_weights_cum, attention_context)
    def initialize_decoder_states(self, memory):
        """디코더 상태 초기화"""
        B = memory.size(0)
        MAX_TIME = memory.size(1)
        
        self.attention_hidden = torch.zeros(B, self.attention_rnn_dim, device=memory.device, dtype=memory.dtype)
        self.attention_cell = torch.zeros(B, self.attention_rnn_dim, device=memory.device, dtype=memory.dtype)
        
        self.decoder_hidden = torch.zeros(B, self.decoder_rnn_dim, device=memory.device, dtype=memory.dtype)
        self.decoder_cell = torch.zeros(B, self.decoder_rnn_dim, device=memory.device, dtype=memory.dtype)
        
        self.attention_weights = torch.zeros(B, MAX_TIME, device=memory.device, dtype=memory.dtype)
        self.attention_weights_cum = torch.zeros(B, MAX_TIME, device=memory.device, dtype=memory.dtype)
        self.attention_context = torch.zeros(B, self.encoder_embedding_dim, device=memory.device, dtype=memory.dtype)
        
        self.memory = memory
        self.processed_memory = self.attention_layer.memory_layer(memory)
        self.mask = None
    
    def parse_decoder_inputs(self, decoder_inputs):
        decoder_inputs = decoder_inputs.view(decoder_inputs.size(0), int(decoder_inputs.size(1) / self.mel_dim), self.mel_dim)
        decoder_inputs = decoder_inputs.transpose(1, 2)
        return decoder_inputs
    
    def parse_decoder_outputs(self, mel_outputs, gate_outputs, alignments):
        alignments = torch.stack(alignments).transpose(0, 1)
        gate_outputs = torch.stack(gate_outputs).transpose(0, 1)
        gate_outputs = gate_outputs.contiguous()
        mel_outputs = torch.stack(mel_outputs).transpose(0, 1).contiguous()
        mel_outputs = mel_outputs.view(mel_outputs.size(0), -1, self.mel_dim)
        mel_outputs = mel_outputs.transpose(1, 2)
        
        return mel_outputs, gate_outputs, alignments
    
    def decode(self, decoder_input):
        cell_input = torch.cat((decoder_input, self.attention_context), -1)
        
        self.attention_hidden, self.attention_cell = self.attention_rnn(cell_input, (self.attention_hidden, self.attention_cell))
        self.attention_hidden = F.dropout(self.attention_hidden, self.p_attention_dropout, self.training)
        
        attention_weights_cat = torch.cat((self.attention_weights.unsqueeze(1), self.attention_weights_cum.unsqueeze(1)), dim=1)
        self.attention_context, self.attention_weights = self.attention_layer(self.attention_hidden, self.memory, self.processed_memory, attention_weights_cat, self.mask)
        
        self.attention_weights_cum += self.attention_weights
        decoder_input = torch.cat((self.attention_hidden, self.attention_context), -1)
        
        self.decoder_hidden, self.decoder_cell = self.decoder_rnn(decoder_input, (self.decoder_hidden, self.decoder_cell))
        self.decoder_hidden = F.dropout(self.decoder_hidden, self.p_decoder_dropout, self.training)
        
        decoder_hidden_attention_context = torch.cat((self.decoder_hidden, self.attention_context), dim=1)
        decoder_output = self.linear_projection(decoder_hidden_attention_context)
        
        gate_prediction = self.gate_layer(decoder_hidden_attention_context)
        
        return decoder_output, gate_prediction, self.attention_weights
    
    # 기존 코드
    # def forward(self, memory, decoder_inputs, memory_lengths):
    #     decoder_input = self.get_go_frame(memory).unsqueeze(1)
    #     decoder_inputs = self.parse_decoder_inputs(decoder_inputs)
    #     decoder_inputs = torch.cat((decoder_input, decoder_inputs), dim=1)
    #     decoder_inputs = self.prenet(decoder_inputs)
        
    #     self.initialize_decoder_states(memory, mask=~get_mask_from_lengths(memory_lengths))
        
    #     mel_outputs, gate_outputs, alignments = [], [], []
    #     while len(mel_outputs) < decoder_inputs.size(1) - 1:
    #         decoder_input = decoder_inputs[:, len(mel_outputs)]
    #         mel_output, gate_output, attention_weights = self.decode(decoder_input)
    #         mel_outputs += [mel_output.squeeze(1)]
    #         gate_outputs += [gate_output.squeeze(1)]
    #         alignments += [attention_weights]
        
    #     mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(mel_outputs, gate_outputs, alignments)
        
    #     return mel_outputs, gate_outputs, alignments
    def forward(self, memory, decoder_inputs, memory_lengths):
        # 간단한 더미 출력으로 테스트
        batch_size = memory.size(0)
        max_mel_len = decoder_inputs.size(1) // self.mel_dim
        
        # 더미 출력 생성
        mel_outputs = torch.randn(batch_size, self.mel_dim, max_mel_len, device=memory.device)
        gate_outputs = torch.zeros(batch_size, max_mel_len, device=memory.device)
        alignments = torch.zeros(batch_size, max_mel_len, memory.size(1), device=memory.device)
        
        return mel_outputs, gate_outputs, alignments

# Tacotron2 메인 모델
class Tacotron2(nn.Module):
    def __init__(self, vocab_size):
        super(Tacotron2, self).__init__()
        
        # 모델 하이퍼파라미터 (경량화를 위해 크기 축소)
        self.embedding_dim = 256  # 원래 512에서 축소
        self.encoder_embedding_dim = 256  # 원래 512에서 축소
        self.encoder_n_convolutions = 3
        self.encoder_kernel_size = 5
        self.attention_rnn_dim = 512  # 원래 1024에서 축소
        self.attention_dim = 64  # 원래 128에서 축소
        self.attention_location_n_filters = 16  # 원래 32에서 축소
        self.attention_location_kernel_size = 31
        self.decoder_rnn_dim = 512  # 원래 1024에서 축소
        self.prenet_dim = 128  # 원래 256에서 축소
        self.max_decoder_steps = 1000
        self.gate_threshold = 0.5
        self.p_attention_dropout = 0.1
        self.p_decoder_dropout = 0.1
        self.postnet_embedding_dim = 256  # 원래 512에서 축소
        self.postnet_kernel_size = 5
        self.postnet_n_convolutions = 5
        self.mel_dim = 80
        safe_vocab_size = max(vocab_size, 100)
        
        # self.embedding = nn.Embedding(vocab_size, self.embedding_dim)
        self.embedding = nn.Embedding(safe_vocab_size, self.embedding_dim)
        std = np.sqrt(2.0 / (safe_vocab_size + self.embedding_dim))
        val = np.sqrt(3.0) * std
        self.embedding.weight.data.uniform_(-val, val)
        
        self.encoder = Encoder(safe_vocab_size, self.embedding_dim, self.encoder_n_convolutions, self.encoder_embedding_dim, self.encoder_kernel_size)
        
        self.decoder = Decoder(self.mel_dim, self.encoder_embedding_dim, self.attention_rnn_dim, self.decoder_rnn_dim, self.attention_dim, self.attention_location_n_filters, self.attention_location_kernel_size, self.prenet_dim, self.max_decoder_steps, self.gate_threshold, self.p_attention_dropout, self.p_decoder_dropout)
        
        self.postnet = Postnet(self.mel_dim, self.postnet_embedding_dim, self.postnet_kernel_size, self.postnet_n_convolutions)
    
    def forward(self, text_inputs, text_lengths, mels, mel_lengths):
        text_lengths, mel_lengths = text_lengths.data, mel_lengths.data
        
        # 이 라인을 삭제하거나 주석 처리하세요.
        # embedded_inputs = self.embedding(text_inputs).transpose(1, 2) 
        
        # text_inputs를 encoder에 직접 전달합니다.
        encoder_outputs = self.encoder(text_inputs, text_lengths)
        
        mel_outputs, gate_outputs, alignments = self.decoder(encoder_outputs, mels, text_lengths)
        
        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet
        
        return mel_outputs, mel_outputs_postnet, gate_outputs, alignments


# def get_mask_from_lengths(lengths):
#     max_len = torch.max(lengths).item()
#     ids = torch.arange(0, max_len, out=torch.cuda.LongTensor(max_len))
#     mask = (ids < lengths.unsqueeze(1)).bool()
#     return mask
def get_mask_from_lengths(lengths):
    max_len = torch.max(lengths).item()
    ids = torch.arange(0, max_len, device=lengths.device)  # device 명시
    mask = (ids < lengths.unsqueeze(1)).bool()
    return mask

# collate_fn 시그니처 수정 및 로직 변경
def collate_fn(batch, vocab_size): # vocab_size를 인자로 받도록 수정
    batch.sort(key=lambda x: x['text_length'], reverse=True)
    
    texts = [item['text'] for item in batch]
    mels = [item['mel'] for item in batch]
    text_lengths = torch.LongTensor([len(item['text']) for item in batch])
    mel_lengths = torch.LongTensor([item['mel'].shape[1] for item in batch])
    
    # 인덱스 검증 (동적으로 vocab_size 사용)
    max_valid_index = vocab_size - 1
    for i, text in enumerate(texts):
        if text.numel() > 0 and text.max() > max_valid_index:
            print(f"경고: 텍스트 {i}에서 범위 초과 인덱스({text.max()})가 어휘사전 크기({vocab_size})를 벗어났습니다. PAD(0)으로 변경합니다.")
            text[text > max_valid_index] = 0  # 범위를 벗어나는 값을 PAD 토큰(0)으로 강제 변환
            texts[i] = text

    # 파이토치 내장 함수를 사용한 효율적인 패딩
    text_padded = torch.nn.utils.rnn.pad_sequence(texts, batch_first=True, padding_value=0)
    
    # 멜 스펙트로그램 패딩
    mel_padded = torch.zeros(len(batch), 80, mel_lengths.max())
    for i, mel in enumerate(mels):
        mel_padded[i, :, :mel.shape[1]] = mel
    
    return text_padded, mel_padded, text_lengths, mel_lengths


# 학습 함수
def train_tacotron2(json_dir, audio_dir, save_dir, epochs=100, batch_size=8, lr=1e-3):
    """Tacotron2 모델 학습 (디렉토리 기반)"""
    from functools import partial
    
    # 디바이스 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"사용 디바이스: {device}")
    
    # 텍스트 프로세서 초기화
    text_processor = KoreanTextProcessor()
    vocab_size = len(text_processor.char_to_id)
    
    # 데이터셋 및 데이터로더 생성
    print(f"JSON 파일 디렉토리: {json_dir}")
    print(f"오디오 파일 디렉토리: {audio_dir}")
    
    dataset = KoreanTTSDataset(json_dir, audio_dir, text_processor)
    dataset.data = dataset.data[:100]  # 처음 10,000개만 사용

    collate_with_vocab = partial(collate_fn, vocab_size=vocab_size)

    
    # dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=0)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                        collate_fn=collate_with_vocab, num_workers=0)
    
    print(f"데이터셋 크기: {len(dataset)}")
    print(f"vocabulary 크기: {vocab_size}")
    
    if len(dataset) == 0:
        print("❌ 오류: 데이터셋이 비어있습니다. JSON 파일과 오디오 파일 경로를 확인해주세요.")
        return None, None
    
    # 데이터 샘플 확인
    print("\n=== 데이터 샘플 확인 ===")
    for i in range(min(3, len(dataset))):
        sample = dataset.data[i]
        print(f"샘플 {i+1}:")
        print(f"  텍스트: {sample['text'][:50]}...")
        print(f"  오디오: {sample['audio']}")
        print(f"  JSON: {sample['json_file']}")
    print("=" * 30)
    
    # 모델 초기화
    model = Tacotron2(vocab_size).to(device)
    
    # 옵티마이저
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
    
    # 손실 함수
    criterion = nn.MSELoss()
    
    # 학습 루프
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        total_batches = len(dataloader)
        for batch_idx, (text_padded, mel_padded, text_lengths, mel_lengths) in enumerate(dataloader):
            text_padded = text_padded.to(device)
            mel_padded = mel_padded.to(device)
            text_lengths = text_lengths.to(device)
            mel_lengths = mel_lengths.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            mel_outputs, mel_outputs_postnet, gate_outputs, alignments = model(
                text_padded, text_lengths, mel_padded, mel_lengths
            )
            
            # 손실 계산
            mel_loss = criterion(mel_outputs, mel_padded)
            mel_postnet_loss = criterion(mel_outputs_postnet, mel_padded)
            gate_loss = nn.BCEWithLogitsLoss()(gate_outputs, torch.zeros_like(gate_outputs))
            
            total_batch_loss = mel_loss + mel_postnet_loss + gate_loss
            
            # Backward pass
            total_batch_loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            total_loss += total_batch_loss.item()
            
            if batch_idx % max(1, total_batches // 10) == 0 or (batch_idx + 1) % 5 == 0:
                progress = (batch_idx + 1) / total_batches * 100
                print(f"📊 Batch {batch_idx+1:3d}/{total_batches} ({progress:5.1f}%) - Loss: {total_batch_loss.item():.4f}")
    
        avg_loss = total_loss / len(dataloader)
        print(f"✅ Epoch {epoch+1}/{epochs} 완료 - 평균 Loss: {avg_loss:.4f}")
        
        scheduler.step()
        
        # 모델 저장 (매 10 에포크마다)
        if (epoch + 1) % 10 == 0:
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
                'text_processor': text_processor
            }
            save_path = os.path.join(save_dir, f'tacotron2_epoch_{epoch+1}.pth')
            os.makedirs(save_dir, exist_ok=True)
            torch.save(checkpoint, save_path)
            print(f'모델 저장: {save_path}')
    
    print("학습 완료!")
    return model, text_processor



# 추론 함수
def inference_tacotron2(model, text_processor, text, device):
    """학습된 모델로 음성 합성"""
    model.eval()
    
    with torch.no_grad():
        # 텍스트 전처리
        text_sequence = text_processor.text_to_sequence(text)
        text_tensor = torch.LongTensor(text_sequence).unsqueeze(0).to(device)
        text_length = torch.LongTensor([len(text_sequence)]).to(device)
        
        # 인코더 통과
        embedded_inputs = model.embedding(text_tensor).transpose(1, 2)
        encoder_outputs = model.encoder.inference(text_tensor)
        
        # 디코더 초기화
        memory = encoder_outputs
        decoder_input = model.decoder.get_go_frame(memory)
        
        (model.decoder.attention_hidden, 
         model.decoder.attention_cell,
         model.decoder.decoder_hidden, 
         model.decoder.decoder_cell,
         model.decoder.attention_weights,
         model.decoder.attention_weights_cum, 
         model.decoder.attention_context) = model.decoder.initialize_decoder_states(memory, None)
        
        model.decoder.memory = memory
        model.decoder.processed_memory = model.decoder.attention_layer.memory_layer(memory)
        model.decoder.mask = None
        
        mel_outputs = []
        gate_outputs = []
        alignments = []
        
        # 디코딩 루프
        while True:
            decoder_input = model.decoder.prenet(decoder_input)
            mel_output, gate_output, attention_weights = model.decoder.decode(decoder_input)
            
            mel_outputs.append(mel_output.squeeze(1))
            gate_outputs.append(gate_output)
            alignments.append(attention_weights)
            
            # 종료 조건 확인
            if torch.sigmoid(gate_output.data) > model.decoder.gate_threshold:
                break
            elif len(mel_outputs) == model.decoder.max_decoder_steps:
                print("최대 디코딩 스텝에 도달했습니다.")
                break
                
            decoder_input = mel_output
        
        # 출력 정리
        mel_outputs = torch.stack(mel_outputs).transpose(0, 1)
        mel_outputs = mel_outputs.transpose(1, 2)
        
        # 포스트넷 적용
        mel_outputs_postnet = model.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet
        
        return mel_outputs_postnet.squeeze(0).cpu().numpy()

# 멜 스펙트로그램을 오디오로 변환 (Griffin-Lim 알고리즘)
def mel_to_audio(mel_spectrogram, sr=22050, n_fft=1024, hop_length=256, win_length=1024, n_iter=50):
    """멜 스펙트로그램을 오디오로 변환"""
    
    # 멜 스펙트로그램을 선형 스펙트로그램으로 변환
    mel_to_linear_matrix = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=mel_spectrogram.shape[0])
    linear_spec = np.dot(mel_to_linear_matrix.T, mel_spectrogram)
    
    # 로그 스케일에서 원래 스케일로 변환
    linear_spec = np.exp(linear_spec) - 1e-9
    
    # Griffin-Lim 알고리즘으로 위상 복원
    audio = librosa.griffinlim(
        linear_spec, 
        n_iter=n_iter, 
        hop_length=hop_length, 
        win_length=win_length
    )
    
    return audio

# 모델 경량화 함수
def quantize_model(model):
    """모델 양자화로 경량화"""
    model.eval()
    quantized_model = torch.quantization.quantize_dynamic(
        model, {nn.Linear, nn.Conv1d, nn.LSTM, nn.LSTMCell}, dtype=torch.qint8
    )
    return quantized_model

# 모델 프루닝 함수  
def prune_model(model, amount=0.2):
    """모델 프루닝으로 경량화"""
    import torch.nn.utils.prune as prune
    
    parameters_to_prune = []
    for module in model.modules():
        if isinstance(module, (nn.Linear, nn.Conv1d)):
            parameters_to_prune.append((module, 'weight'))
    
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=amount,
    )
    
    # 프루닝된 가중치를 영구적으로 제거
    for module, param in parameters_to_prune:
        prune.remove(module, param)
    
    return model

# 디렉토리 스캔 및 데이터 분석 함수
def analyze_dataset_directory(json_dir, audio_dir=None):
    """데이터셋 디렉토리 분석 및 구조 파악"""
    print(f"📂 디렉토리 분석 시작: {json_dir}")
    print("=" * 60)
    
    if not os.path.exists(json_dir):
        print(f"❌ 오류: 디렉토리가 존재하지 않습니다: {json_dir}")
        return
    
    # JSON 파일 찾기
    json_files = []
    audio_files = []
    
    print("🔍 파일 스캔 중...")
    file_count = 0
    
    for root, dirs, files in os.walk(json_dir):
        for file in files:
            file_count += 1
            if file_count % 5000 == 0:  # 5000개마다 진행상황 표시
                print(f"📂 스캔 중... {file_count:,}개 파일 확인")
                
            if file.lower().endswith('.json'):
                json_files.append(os.path.join(root, file))
            elif file.lower().endswith(('.wav', '.mp3', '.flac', '.m4a')):
                audio_files.append(os.path.join(root, file))
    
    # 오디오 파일 찾기 (원천데이터 경로에서)
    print("🎵 오디오 파일 스캔 중...")
    audio_search_dir = json_dir.replace('라벨링데이터', '원천데이터')
    if 'TL22' in audio_search_dir:
        audio_search_dir = audio_search_dir.replace('TL22', 'TS22')

    if os.path.exists(audio_search_dir):
        audio_file_count = 0
        for root, dirs, files in os.walk(audio_search_dir):
            for file in files:
                audio_file_count += 1
                if audio_file_count % 5000 == 0:
                    print(f"🎵 오디오 스캔 중... {audio_file_count:,}개 파일 확인")
                    
                if file.lower().endswith(('.wav', '.mp3', '.flac', '.m4a')):
                    audio_files.append(os.path.join(root, file))
    
    print(f"📄 JSON 파일 수: {len(json_files):,}")
    print(f"🎵 오디오 파일 수: {len(audio_files):,}")
    
    # 샘플 JSON 파일 구조 분석
    if json_files:
        print(f"\n🔍 샘플 JSON 파일 구조 분석:")
        sample_json = json_files[0]
        print(f"샘플 파일: {sample_json}")
        
        try:
            with open(sample_json, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            print("JSON 구조:")
            print(json.dumps(data, ensure_ascii=False, indent=2)[:500] + "...")
            
            # TransLabelText 찾기
            def find_trans_label_text(obj, path=""):
                results = []
                if isinstance(obj, dict):
                    for key, value in obj.items():
                        current_path = f"{path}.{key}" if path else key
                        if key == "TransLabelText" and isinstance(value, str):
                            results.append((current_path, value))
                        elif isinstance(value, (dict, list)):
                            results.extend(find_trans_label_text(value, current_path))
                elif isinstance(obj, list):
                    for i, item in enumerate(obj):
                        current_path = f"{path}[{i}]"
                        results.extend(find_trans_label_text(item, current_path))
                return results
            
            trans_texts = find_trans_label_text(data)
            if trans_texts:
                print(f"\n✅ TransLabelText 발견:")
                for path, text in trans_texts:
                    print(f"  경로: {path}")
                    print(f"  텍스트: {text[:50]}...")
            else:
                print(f"\n❌ TransLabelText를 찾을 수 없습니다.")
                
        except Exception as e:
            print(f"JSON 파일 읽기 오류: {e}")
    
    # 오디오 파일 분포 확인
    if audio_files:
        print(f"\n🎵 오디오 파일 분포:")
        audio_extensions = {}
        for audio_file in audio_files[:100]:  # 처음 100개만 확인
            ext = os.path.splitext(audio_file)[1].lower()
            audio_extensions[ext] = audio_extensions.get(ext, 0) + 1
        
        for ext, count in audio_extensions.items():
            print(f"  {ext}: {count}개")
    
    # 매칭되는 JSON-오디오 쌍 확인
    print(f"\n🔗 JSON-오디오 매칭 확인:")
    matched_pairs = 0
    for json_file in json_files[:10]:  # 처음 10개만 확인
        json_name = os.path.splitext(os.path.basename(json_file))[0]
        json_dir_path = os.path.dirname(json_file)
        
        # 개선된 오디오 경로 찾기 로직 사용
        audio_path = None
        
        # 1. 같은 디렉토리에서 찾기
        audio_path = os.path.join(json_dir_path, json_name + '.wav')
        if os.path.exists(audio_path):
            matched_pairs += 1
            print(f"  ✅ 매칭 성공: {os.path.basename(json_file)}")
            continue
        
        # 2. 라벨링데이터 -> 원천데이터로 경로 변경
        if '라벨링데이터' in json_dir_path:
            audio_dir_path = json_dir_path.replace('라벨링데이터', '원천데이터')
            # TL22 -> TS22 변경
            if 'TL22' in audio_dir_path:
                audio_dir_path = audio_dir_path.replace('TL22', 'TS22')
            
            audio_path = os.path.join(audio_dir_path, json_name + '.wav')
            if os.path.exists(audio_path):
                matched_pairs += 1
                print(f"  ✅ 매칭 성공: {os.path.basename(json_file)} -> 원천데이터")
                continue
        
        print(f"  ❌ 매칭 실패: {os.path.basename(json_file)}")

    print(f"  매칭된 쌍: {matched_pairs}/{min(10, len(json_files))} (샘플 10개 중)")
    
    print("=" * 60)
    print("💡 분석 완료!")
    
    return {
        'json_files': len(json_files),
        'audio_files': len(audio_files), 
        'sample_json': json_files[0] if json_files else None
    }


# 사용 예시 (업데이트)
def main():
    # 실제 데이터 경로 설정
    json_dir = r"D:\workspace\TTS_prj\datasets\014.다화자 음성합성 데이터\01.데이터\1.Training\라벨링데이터\TL22\TL22\2.여성\2400문장"
    audio_dir = r"D:\workspace\TTS_prj\datasets\014.다화자 음성합성 데이터\01.데이터\1.Training\원천데이터\TS22\TS22\2.여성\2400문장"
    save_dir = r"D:\workspace\TTS_prj\models"
    
    # 1. 데이터셋 분석
    print("🔍 데이터셋 분석 중...")
    analyze_dataset_directory(json_dir, audio_dir)
    
    # 2. 학습 진행 여부 확인
    proceed = input("\n학습을 진행하시겠습니까? (y/n): ")
    if proceed.lower() != 'y':
        print("학습을 취소합니다.")
        return
    
    # 3. 텍스트 프로세서와 데이터셋 생성 (검증을 위해 미리 생성)
    text_processor = KoreanTextProcessor()
    dataset = KoreanTTSDataset(json_dir, audio_dir, text_processor)
    dataset.data = dataset.data[:100]  # 테스트용으로 축소
    
    # 4. 인덱스 범위 검증
    print("=== 인덱스 범위 검증 ===")
    vocab_size = len(text_processor.char_to_id)
    print(f"Vocabulary 크기: {vocab_size}")
    
    # 샘플 데이터로 인덱스 확인
    sample_texts = [dataset[i]['text'] for i in range(min(10, len(dataset)))]
    max_indices = [text.max().item() for text in sample_texts]
    print(f"실제 최대 인덱스들: {max_indices}")
    print(f"모든 인덱스가 vocabulary 범위 내인가: {all(idx < vocab_size for idx in max_indices)}")
    
    if any(idx >= vocab_size for idx in max_indices):
        print("오류: 인덱스가 vocabulary 범위를 초과합니다!")
        return
    
    # 5. 학습 시작
    print("🚀 Tacotron2 모델 학습 시작...")
    model, text_processor = train_tacotron2(
        json_dir=json_dir,
        audio_dir=audio_dir,
        save_dir=save_dir,
        epochs=10,
        batch_size=1,  # 1로 축소
        lr=1e-3
    )
    
    
    if model is None:
        print("❌ 학습 실패!")
        return
    
    # 4. 모델 경량화
    print("⚡ 모델 경량화 중...")
    model = prune_model(model, amount=0.3)
    quantized_model = quantize_model(model)
    
    # 5. 최종 모델 저장
    final_save_path = os.path.join(save_dir, 'tacotron2_lightweight_final.pth')
    os.makedirs(save_dir, exist_ok=True)
    torch.save({
        'model_state_dict': quantized_model.state_dict(),
        'text_processor': text_processor
    }, final_save_path)
    print(f"✅ 경량화된 최종 모델 저장: {final_save_path}")
    
    # 6. 추론 테스트
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    test_text = "안녕하세요. 한국어 음성합성 테스트입니다."
    
    print(f"🎤 추론 테스트: '{test_text}'")
    mel_output = inference_tacotron2(quantized_model, text_processor, test_text, device)
    
    # 7. 오디오 생성 및 저장
    audio = mel_to_audio(mel_output)
    
    import soundfile as sf
    output_audio_path = os.path.join(save_dir, 'output_test.wav')
    sf.write(output_audio_path, audio, 22050)
    print(f"🔊 생성된 오디오 저장: {output_audio_path}")

# 간단한 데이터셋 분석만 실행하는 함수
def quick_analysis():
    """빠른 데이터셋 분석"""
    json_dir = r"D:\workspace\TTS_prj\datasets\014.다화자 음성합성 데이터\01.데이터\1.Training\라벨링데이터\TL22\TL22\2.여성\2400문장"
    analyze_dataset_directory(json_dir)

# 모델 로드 및 사용 함수
def load_and_use_model(model_path, text):
    """저장된 모델 로드하여 음성 합성"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 체크포인트 로드
    checkpoint = torch.load(model_path, map_location=device)
    text_processor = checkpoint['text_processor']
    
    # 모델 초기화 및 가중치 로드
    vocab_size = len(text_processor.char_to_id)
    model = Tacotron2(vocab_size).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # 추론
    mel_output = inference_tacotron2(model, text_processor, text, device)
    audio = mel_to_audio(mel_output)
    
    return audio

# JSON 파일 구조 확인 함수
def check_json_structure(json_file):
    """JSON 파일의 구조를 확인하여 TransLabelText 위치 파악"""
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    def find_trans_label_text(obj, path=""):
        """재귀적으로 TransLabelText 찾기"""
        if isinstance(obj, dict):
            for key, value in obj.items():
                current_path = f"{path}.{key}" if path else key
                if key == "TransLabelText":
                    print(f"TransLabelText 발견: {current_path} = {value}")
                elif isinstance(value, (dict, list)):
                    find_trans_label_text(value, current_path)
        elif isinstance(obj, list):
            for i, item in enumerate(obj):
                current_path = f"{path}[{i}]"
                find_trans_label_text(item, current_path)
    
    print(f"JSON 파일 구조 분석: {json_file}")
    find_trans_label_text(data)
    print("-" * 50)

if __name__ == "__main__":
    # 실행 옵션 선택
    print("🎯 실행 옵션을 선택하세요:")
    print("1. 데이터셋 분석만 실행")
    print("2. 전체 학습 파이프라인 실행")
    
    choice = input("선택 (1 또는 2): ")
    
    if choice == "1":
        quick_analysis()
    elif choice == "2":
        main()
    else:
        print("잘못된 선택입니다. 데이터셋 분석을 실행합니다.")
        quick_analysis()